In [8]:
import pickle
project = 'openstack'
with open('../resource/'+project+'_train.pkl', 'rb') as f_train, open('../resource/'+project+'_test.pkl', 'rb') as f_test:
        tr_data = pickle.load(f_train)
        te_data = pickle.load(f_test)
pred_list = []
kinds = ['train', 'test']
models = ['lgb', 'code_cnn', 'msg_tf']
for kind in kinds:
    for model in models:
        with open('../pred/'+project+'-'+model+'-'+'random'+'-'+kind+'.pkl', 'rb') as f:
            pred_list.append(pickle.load(f))
tr_pred = [[pred_list[0][j], pred_list[1][j], pred_list[2][j]] for j in range(len(pred_list[0]))]
te_pred = [[pred_list[3][j], pred_list[4][j], pred_list[5][j]] for j in range(len(pred_list[3]))]                
tr_data = [tr_pred, tr_data[5]]
te_data = [te_pred, te_data[5]]

In [9]:
tr_data[0]

[[0.16092745326840102, 0.021402199, 0.509486],
 [0.3193696255443434, 0.07866127, 0.12485164],
 [0.8729744960587329, 0.91129243, 0.28266403],
 [0.3231291645480817, 0.24457371, 0.5229826],
 [0.281422419653164, 0.8578265, 0.043312617],
 [0.8073646044152452, 0.8830279, 0.37239048],
 [0.5349141448498056, 0.7582897, 0.10247494],
 [0.674647812381105, 0.84463984, 0.030646747],
 [0.2248353561809572, 0.33007056, 0.015908524],
 [0.5057495434834242, 0.5791738, 0.056447506],
 [0.7086046771656842, 0.5558742, 0.80761737],
 [0.8597510138769027, 0.8156691, 0.807817],
 [0.5618877600072832, 0.889421, 0.70573884],
 [0.39595586424323964, 0.54397607, 0.64133734],
 [0.5027521829807249, 0.79465795, 0.11872009],
 [0.4199598667761703, 0.71378154, 0.096476026],
 [0.27730186310120186, 0.25220284, 0.6098979],
 [0.16946049755327852, 0.41079703, 0.015940268],
 [0.790287072381172, 0.89598703, 0.68607515],
 [0.10430006261034865, 0.09597076, 0.03436511],
 [0.39989704769619694, 0.27365875, 0.5214928],
 [0.54352836797843

In [10]:
from optuna.integration import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils.class_weight import compute_sample_weight

In [11]:
sample_weights = compute_sample_weight(class_weight='balanced', y=tr_data[1])
# _weights = compute_class_weight(class_weight='balanced', classes=np.unique(tr_dataset[5]), y=tr_dataset[5])

In [12]:

lgb_train = lgb.Dataset(np.array(tr_data[0]), np.array(tr_data[1]), weight=sample_weights)

In [13]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
lgb_params = {
    'objective':'binary',
    'metric': 'binary_logloss',
    'verbosity':-1,
}
tuner_cv = lgb.LightGBMTunerCV(
    lgb_params, lgb_train,
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True),
                lgb.log_evaluation(100)],
    return_cvbooster=True,
    folds=folds
)

tuner_cv.run()

[I 2023-12-02 15:43:06,546] A new study created in memory with name: no-name-e255f974-cdc2-4153-a0cc-89c52fb534fe
feature_fraction, val_score: 0.538217:  14%|#4        | 1/7 [00:00<00:01,  5.61it/s]

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[24]	cv_agg's valid binary_logloss: 0.538217 + 0.0155875
Training until validation scores don't improve for 10 rounds


feature_fraction, val_score: 0.532583:  43%|####2     | 3/7 [00:00<00:00,  6.35it/s]

Early stopping, best iteration is:
[24]	cv_agg's valid binary_logloss: 0.538217 + 0.0155875
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[22]	cv_agg's valid binary_logloss: 0.532583 + 0.0115015
Training until validation scores don't improve for 10 rounds


feature_fraction, val_score: 0.531465:  71%|#######1  | 5/7 [00:00<00:00,  7.39it/s]

...

Early stopping, best iteration is:
[85]	cv_agg's valid binary_logloss: 0.52549 + 0.0115399
Training until validation scores don't improve for 10 rounds


min_child_samples, val_score: 0.525490:  80%|########  | 4/5 [00:00<00:00,  8.14it/s]

Early stopping, best iteration is:
[85]	cv_agg's valid binary_logloss: 0.52549 + 0.0115399
Training until validation scores don't improve for 10 rounds


min_child_samples, val_score: 0.525472: 100%|##########| 5/5 [00:00<00:00,  8.11it/s]

Early stopping, best iteration is:
[85]	cv_agg's valid binary_logloss: 0.525472 + 0.0114942


In [15]:
#openstack logloss 不均衡対策
print(f'Best score: {tuner_cv.best_score}')
print('Best params:')
print(tuner_cv.best_params)
model = tuner_cv.get_best_booster()
pred = model.predict(np.array(te_data[0]), num_iteration=model.best_iteration)
mean_pred = np.mean(pred, axis=0)
print(mean_pred)
print(roc_auc_score(te_data[1], mean_pred))
print(log_loss(te_data[1], mean_pred))

Best score: 0.5254720652267872
Best params:
{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 9.437038741213511, 'lambda_l2': 1.1639547844209527e-08, 'num_leaves': 4, 'feature_fraction': 0.4, 'bagging_fraction': 0.9953340015239459, 'bagging_freq': 1, 'min_child_samples': 100}
[0.56411557 0.8885075  0.21089981 ... 0.45397972 0.09104037 0.17435259]
0.8158902121697358
0.6287841213707461


In [7]:
#qt logloss 不均衡対策
print(f'Best score: {tuner_cv.best_score}')
print('Best params:')
print(tuner_cv.best_params)
model = tuner_cv.get_best_booster()
pred = model.predict(np.array(te_data[0]), num_iteration=model.best_iteration)
mean_pred = np.mean(pred, axis=0)
print(mean_pred)
print(roc_auc_score(te_data[1], mean_pred))
print(log_loss(te_data[1], mean_pred))

Best score: 0.5280279874638432
Best params:
{'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'feature_pre_filter': False, 'lambda_l1': 0.0011915139883967448, 'lambda_l2': 3.4588147191727472e-06, 'num_leaves': 5, 'feature_fraction': 1.0, 'bagging_fraction': 0.7929649221365002, 'bagging_freq': 3, 'min_child_samples': 25}
[0.33271095 0.53479266 0.10296248 ... 0.71196917 0.59963328 0.11965684]
0.7786629395679638
0.5453967647586538
